In [68]:
import numpy as np
from gbp.gbp import FactorGraph, VariableNode, Factor

# -----------------------
# SLAM-like base graph
# -----------------------
def make_slam_like_graph(N=100, step_size=25, loop_prob=0.05, loop_radius=50, prior_prop=0.0, rng=None,):
    nodes, edges = [], []
    positions = []
    x, y = 0.0, 0.0
    positions.append((x, y))
    for _ in range(1, int(N)):
        dx, dy = np.random.randn(2)
        norm = np.sqrt(dx**2 + dy**2) + 1e-6
        dx, dy = dx / norm * float(step_size), dy / norm * float(step_size)
        x, y = x + dx, y + dy
        positions.append((x, y))
    for i, (px, py) in enumerate(positions):
        nodes.append({
            "data": {"id": f"{i}", "layer": 0, "dim": 2},
            "position": {"x": float(px), "y": float(py)}
        })
    for i in range(int(N) - 1):
        edges.append({"data": {"source": f"{i}", "target": f"{i+1}"}})
    for i in range(int(N)):
        for j in range(i + 5, int(N)):
            if np.random.rand() < float(loop_prob):
                xi, yi = positions[i]; xj, yj = positions[j]
                if np.hypot(xi-xj, yi-yj) < float(loop_radius):
                    edges.append({"data": {"source": f"{i}", "target": f"{j}"}})

    # 确定强先验的节点集合
    if rng is None:
        rng = np.random.default_rng()

    if prior_prop <= 0.0:
        strong_ids = {0}
    elif prior_prop >= 1.0:
        strong_ids = set(range(N))
    else:
        k = max(1, int(np.floor(prior_prop * N)))
        strong_ids = set(rng.choice(N, size=k, replace=False).tolist())

    # 给 strong prior 节点加 edge
    for i in strong_ids:
        edges.append({
            "data": {"source": f"{i}", "target": "prior"}
        })


    return nodes, edges

# -----------------------
# GBP Graph 构建
# -----------------------
def build_noisy_pose_graph(
    nodes,
    edges,
    prior_sigma: float = 10,
    odom_sigma: float = 10,
    tiny_prior: float = 1e-10,
    rng=None,
):
    
    """
    构造二维 pose-only 因子图（线性，高斯），并注入噪声。
    参数:
      prior_sigma : 强先验的标准差（小=强）
      odom_sigma  : 里程计测量噪声标准差
      prior_prop  : 0.0=仅 anchor；(0,1)=按比例随机选；>=1.0=全体
      tiny_prior  : 所有节点默认加的极小先验，防止奇异
      seed        : 随机种子（可复现）
    """

    fg = FactorGraph(nonlinear_factors=False, eta_damping=0)

    var_nodes = []
    I2 = np.eye(2, dtype=float)
    N = len(nodes)

    # ---- 预生成噪声 ----
    prior_noises = {}
    odom_noises = {}

    if rng is None:
        rng = np.random.default_rng()

    # 为所有边生成噪声
    for e in edges:
        src = e["data"]["source"]; dst = e["data"]["target"]
        # 二元边
        if dst != "prior":
            odom_noises[(int(src[:]), int(dst[:]))] = rng.normal(0.0, odom_sigma, size=2)
        # 一元边（强先验）
        elif dst == "prior":
            prior_noises[int(src[:])] = rng.normal(0.0, prior_sigma, size=2)


    # ---- variable nodes ----
    for i, n in enumerate(nodes):
        v = VariableNode(i, dofs=2)
        v.GT = np.array([n["position"]["x"], n["position"]["y"]], dtype=float)

        # 极小先验
        v.prior.lam = tiny_prior * I2
        v.prior.eta = np.zeros(2, dtype=float)

        var_nodes.append(v)

    fg.var_nodes = var_nodes
    fg.n_var_nodes = len(var_nodes)


    # ---- prior factors ----
    def meas_fn_unary(x, *args):
        return x
    def jac_fn_unary(x, *args):
        return np.eye(2)
    # ---- odometry factors ----
    def meas_fn(xy, *args):
        return xy[2:] - xy[:2]
    def jac_fn(xy, *args):
        return np.array([[-1, 0, 1, 0],
                         [ 0,-1, 0, 1]], dtype=float)
    
    factors = []
    fid = 0

    for e in edges:
        src = e["data"]["source"]; dst = e["data"]["target"]
        if dst != "prior":
            i, j = int(src[:]), int(dst[:])
            vi, vj = var_nodes[i], var_nodes[j]

            meas = (vj.GT - vi.GT) + odom_noises[(i, j)]

            f = Factor(fid, [vi, vj], meas, odom_sigma, meas_fn, jac_fn)
            f.type = "base"
            linpoint = np.r_[vi.GT, vj.GT]
            f.compute_factor(linpoint=linpoint, update_self=True)

            factors.append(f)
            vi.adj_factors.append(f)
            vj.adj_factors.append(f)
            fid += 1

        else:
            i = int(src[:])
            vi = var_nodes[i]
            z = vi.GT + prior_noises[i]

            f = Factor(fid, [vi], z, prior_sigma, meas_fn_unary, jac_fn_unary)
            f.type = "prior"
            f.compute_factor(linpoint=z, update_self=True)

            factors.append(f)
            vi.adj_factors.append(f)
            fid += 1

    fg.factors = factors
    fg.n_factor_nodes = len(factors)
    return fg


In [69]:
# -----------------------
# K-Means 聚合
# -----------------------
def fuse_to_super_knn(prev_nodes, prev_edges, k, layer_idx, max_iters=20, tol=1e-6, seed=0):
    positions = np.array([[n["position"]["x"], n["position"]["y"]] for n in prev_nodes], dtype=float)
    n = positions.shape[0]
    if k <= 0: k = 1
    k = min(k, n)
    rng = np.random.default_rng(seed)

    # k-means++ init
    centers = np.empty((k, 2), dtype=float)
    first = rng.integers(0, n)
    centers[0] = positions[first]
    closest_d2 = np.sum((positions - centers[0])**2, axis=1)
    for i in range(1, k):
        denom = float(closest_d2.sum()) + 1e-12
        probs = closest_d2 / denom
        idx = rng.choice(n, p=probs)
        centers[i] = positions[idx]
        d2_new = np.sum((positions - centers[i])**2, axis=1)
        closest_d2 = np.minimum(closest_d2, d2_new)

    # Lloyd iterations
    for _ in range(max_iters):
        d2 = ((positions[:, None, :] - centers[None, :, :]) ** 2).sum(axis=2)
        assign = np.argmin(d2, axis=1)
        moved = 0.0
        for ci in range(k):
            idxs = np.where(assign == ci)[0]
            if idxs.size == 0:
                far_idx = int(np.argmax(np.min(d2, axis=1)))
                new_c = positions[far_idx]
            else:
                new_c = positions[idxs].mean(axis=0)
            moved = max(moved, float(np.linalg.norm(new_c - centers[ci])))
            centers[ci] = new_c
        if moved < tol:
            break

    # final assign
    d2 = ((positions[:, None, :] - centers[None, :, :]) ** 2).sum(axis=2)
    assign = np.argmin(d2, axis=1)

    super_nodes, node_map = [], {}
    for ci in range(k):
        idxs = np.where(assign == ci)[0]
        if idxs.size == 0:
            continue
        pts = positions[idxs]
        mean_x, mean_y = pts.mean(axis=0)
        child_dims = [prev_nodes[i]["data"]["dim"] for i in idxs]
        dim_val = int(max(1, sum(child_dims)))
        nid = f"{ci}"
        super_nodes.append({
            "data": {"id": nid, "layer": layer_idx, "dim": dim_val},
            "position": {"x": float(mean_x), "y": float(mean_y)}
        })
        for i in idxs:
            node_map[prev_nodes[i]["data"]["id"]] = nid
    super_edges, seen = [], set()
    for e in prev_edges:
        u, v = e["data"]["source"], e["data"]["target"]
        if v != "prior":
            su, sv = node_map[u], node_map[v]
            if su != sv:
                eid = tuple(sorted((su, sv)))
                if eid not in seen:
                    super_edges.append({"data": {"source": su, "target": sv}})
                    seen.add(eid)
            elif su == sv:
                eid = tuple(sorted((su, su)))
                if eid not in seen:
                    super_edges.append({"data": {"source": su, "target": "prior"}})
                    seen.add(eid)

        elif v == "prior":
            eid = tuple(sorted((su, su)))
            if eid not in seen:
                super_edges.append({"data": {"source": su, "target": "prior"}})
                seen.add(eid)

    return super_nodes, super_edges, node_map

In [70]:
def build_super_graph(layers):
    """
    基于 layers[-2] 的 base graph, 和 layers[-1] 的 super 分组，构造 super graph。
    要求: layers[-2]["graph"] 已经是构建好的基图（含 unary/binary 因子）。
    layers[-1]["node_map"]: { base_node_id(str, 如 'b12') -> super_node_id(str) }
    """
    # ---------- 取出 base & super ----------
    base_graph = layers[-2]["graph"]
    super_nodes = layers[-1]["nodes"]
    super_edges = layers[-1]["edges"]
    node_map    = layers[-1]["node_map"]   # 'bN' -> 'sX_...'

    # base: id(int)->VariableNode，方便查 dofs 和 mu
    id2var = {vn.variableID: vn for vn in base_graph.var_nodes}

    # ---------- super_id -> [base_id(int)] ----------
    super_groups = {}
    for b_str, s_id in node_map.items():
        b_int = int(b_str)
        super_groups.setdefault(s_id, []).append(b_int)


    # ---------- 为每个 super 组建立 (start, dofs) 表 ----------
    # local_idx[sid][bid] = (start, dofs), total_dofs[sid] = sum(dofs)
    local_idx   = {}
    total_dofs  = {}
    for sid, group in super_groups.items():
        off = 0
        local_idx[sid] = {}
        for bid in group:
            d = id2var[bid].dofs
            local_idx[sid][bid] = (off, d)
            off += d
        total_dofs[sid] = off


    # ---------- 创建 super VariableNodes ----------
    fg = FactorGraph(nonlinear_factors=False, eta_damping=0)

    super_var_nodes = {}
    for i, sn in enumerate(super_nodes):
        sid = sn["data"]["id"]
        dofs = total_dofs.get(sid, 0)

        v = VariableNode(i, dofs=dofs)

        # === 叠加 base GT ===
        gt_vec = np.zeros(dofs)
        for bid, (st, d) in local_idx[sid].items():
            gt_base = getattr(id2var[bid], "GT", None)
            if gt_base is None or len(gt_base) != d:
                gt_base = np.zeros(d)
            gt_vec[st:st+d] = gt_base
        v.GT = gt_vec
        v.prior.lam = 1e-10 * np.eye(dofs, dtype=float)
        v.prior.eta = np.zeros(dofs, dtype=float)

        super_var_nodes[sid] = v
        fg.var_nodes.append(v)


    fg.n_var_nodes = len(fg.var_nodes)

    # ---------- 工具：拼接某组的 linpoint（用 base belief 均值） ----------
    def make_linpoint_for_group(sid):
        x = np.zeros(total_dofs[sid])
        for bid, (st, d) in local_idx[sid].items():
            mu = getattr(id2var[bid], "mu", None)
            if mu is None or len(mu) != d:
                mu = np.zeros(d)
            x[st:st+d] = mu
        return x

    # ---------- 3) super prior（组内 unary + 组内 binary） ----------
    def make_super_prior_factor(sid, base_factors):
        group = super_groups[sid]
        idx_map = local_idx[sid]
        ncols = total_dofs[sid]

        # 选出：所有变量都在组内的因子（unary 或 binary）
        in_group = []
        for f in base_factors:
            vids = [v.variableID for v in f.adj_var_nodes]
            if all(vid in group for vid in vids):
                in_group.append(f)

        def meas_fn_super_prior(x_super, *args):
            meas_fn = []
            for f in in_group:
                vids = [v.variableID for v in f.adj_var_nodes]
                # 拼本因子的局部 x
                x_loc_list = []
                for vid in vids:
                    st, d = idx_map[vid]
                    x_loc_list.append(x_super[st:st+d])
                x_loc = np.concatenate(x_loc_list) if x_loc_list else np.zeros(0)
                meas_fn.append(f.meas_fn(x_loc))
            return np.concatenate(meas_fn) if meas_fn else np.zeros(0)

        def jac_fn_super_prior(x_super, *args):
            Jrows = []
            for f in in_group:
                vids = [v.variableID for v in f.adj_var_nodes]
                # 构造本因子的局部 x，用于（潜在）非线性雅可比
                x_loc_list = []
                dims = []
                for vid in vids:
                    st, d = idx_map[vid]
                    dims.append(d)
                    x_loc_list.append(x_super[st:st+d])
                x_loc = np.concatenate(x_loc_list) if x_loc_list else np.zeros(0)

                Jloc = f.jac_fn(x_loc)
                # 将 Jloc 列块映射回 super 变量的列
                row = np.zeros((Jloc.shape[0], ncols))
                c0 = 0
                for vid, d in zip(vids, dims):
                    st, _ = idx_map[vid]
                    row[:, st:st+d] = Jloc[:, c0:c0+d]
                    c0 += d

                row *= f.gauss_noise_std
                
                Jrows.append(row)
            return np.vstack(Jrows) if Jrows else np.zeros((0, ncols))

        # z_super：拼各 base 因子的 z
        z_list = [f.measurement for f in in_group]
        z_super = np.concatenate(z_list) if z_list else np.zeros(0)

        return meas_fn_super_prior, jac_fn_super_prior, z_super

    # ---------- 4) super between（只取跨组 binary） ----------
    def make_super_between_factor(sidA, sidB, base_factors):
        groupA, groupB = super_groups[sidA], super_groups[sidB]
        idxA, idxB = local_idx[sidA], local_idx[sidB]
        nA, nB = total_dofs[sidA], total_dofs[sidB]

        cross = []
        for f in base_factors:
            vids = [v.variableID for v in f.adj_var_nodes]
            if len(vids) != 2:
                continue
            i, j = vids
            # 恰有一端在 A，一端在 B
            if (i in groupA and j in groupB) or (i in groupB and j in groupA):
                cross.append(f)

        def meas_fn_super_between(xAB, *args):
            xA, xB = xAB[:nA], xAB[nA:]
            meas_fn = []
            for f in cross:
                i, j = [v.variableID for v in f.adj_var_nodes]
                if i in groupA:
                    si, di = idxA[i]
                    sj, dj = idxB[j]
                    xi = xA[si:si+di]
                    xj = xB[sj:sj+dj]
                else:
                    si, di = idxB[i]
                    sj, dj = idxA[j]
                    xi = xB[si:si+di]
                    xj = xA[sj:sj+dj]
                x_loc = np.concatenate([xi, xj])
                meas_fn.append(f.meas_fn(x_loc))
            return np.concatenate(meas_fn) if meas_fn else np.zeros(0)

        def jac_fn_super_between(xAB, *args):
            xA, xB = xAB[:nA], xAB[nA:]
            Jrows = []
            for f in cross:
                i, j = [v.variableID for v in f.adj_var_nodes]
                if i in groupA:
                    si, di = idxA[i]
                    sj, dj = idxB[j]
                    xi = xA[si:si+di]
                    xj = xB[sj:sj+dj]
                    left_start, right_start = si, nA + sj
                else:
                    si, di = idxB[i]
                    sj, dj = idxA[j]
                    xi = xB[si:si+di]
                    xj = xA[sj:sj+dj]
                    left_start, right_start = nA + si, sj
                x_loc = np.concatenate([xi, xj])
                Jloc = f.jac_fn(x_loc)

                row = np.zeros((Jloc.shape[0], nA + nB))
                row[:, left_start:left_start+di]   = Jloc[:, :di]
                row[:, right_start:right_start+dj] = Jloc[:, di:di+dj]

                row *= f.gauss_noise_std
                Jrows.append(row)
            return np.vstack(Jrows) if Jrows else np.zeros((0, nA + nB))

        z_list = [f.measurement for f in cross]
        z_super = np.concatenate(z_list) if z_list else np.zeros(0)

        return meas_fn_super_between, jac_fn_super_between, z_super


    for e in super_edges:
        u, v = e["data"]["source"], e["data"]["target"]

        if v == "prior":
            meas_fn, jac_fn, z = make_super_prior_factor(u, base_graph.factors)
            f = Factor(len(fg.factors), [super_var_nodes[u]], z, 1, meas_fn, jac_fn)
            f.type = "super_prior"
            lin0 = make_linpoint_for_group(u)
            f.compute_factor(linpoint=lin0, update_self=True)
            fg.factors.append(f)
            super_var_nodes[u].adj_factors.append(f)
            
        else:
            continue
            meas_fn, jac_fn, z = make_super_between_factor(u, v, base_graph.factors)
            f = Factor(len(fg.factors), [super_var_nodes[u], super_var_nodes[v]], z, 1, meas_fn, jac_fn)
            f.type = "super_between"
            lin0 = np.concatenate([make_linpoint_for_group(u), make_linpoint_for_group(v)])
            f.compute_factor(linpoint=lin0, update_self=True)
            fg.factors.append(f)
            super_var_nodes[u].adj_factors.append(f)
            super_var_nodes[v].adj_factors.append(f)


    fg.n_factor_nodes = len(fg.factors)
    return fg


In [71]:
N=50
step=25
prob=0.1
radius=50 
prior_prop=1
prior_sigma=1.0
odom_sigma=1.0
layers = []


# -----------------------
# 初始化 & 边界
# -----------------------
def init_layers(N=10, step_size=25, loop_prob=0.05, loop_radius=50, prior_prop=0.0):
    base_nodes, base_edges = make_slam_like_graph(N, step_size, loop_prob, loop_radius, prior_prop)
    return [{"name": "base", "nodes": base_nodes, "edges": base_edges}]

layers = init_layers(N=N, step_size=step, loop_prob=prob, loop_radius=radius, prior_prop=prior_prop)
pair_idx = 0



# 构建 GBP 图
gbp_graph = build_noisy_pose_graph(layers[0]["nodes"], layers[0]["edges"],
                                    prior_sigma=prior_sigma,
                                    odom_sigma=odom_sigma
                                    )
layers[0]["graph"] = gbp_graph
opts=[{"label":"base","value":"base"}]


last = layers[-1]
super_layer_idx = 1

super_nodes, super_edges, node_map = fuse_to_super_knn(last["nodes"], last["edges"], 40, super_layer_idx)
layers.append({"name":f"super{1}", "nodes":super_nodes, "edges":super_edges, "node_map":node_map})
layers[-1]["graph"] = build_super_graph(layers)




In [72]:
layers[-1]["nodes"]

[{'data': {'id': '0', 'layer': 1, 'dim': 2},
  'position': {'x': 89.22054147575263, 'y': -106.37722118329346}},
 {'data': {'id': '1', 'layer': 1, 'dim': 2},
  'position': {'x': -80.4549391619026, 'y': -23.499762332792898}},
 {'data': {'id': '2', 'layer': 1, 'dim': 6},
  'position': {'x': 3.4846054454553013, 'y': 6.487988200612009}},
 {'data': {'id': '3', 'layer': 1, 'dim': 2},
  'position': {'x': -14.457949385874969, 'y': -20.207279300950116}},
 {'data': {'id': '4', 'layer': 1, 'dim': 2},
  'position': {'x': 16.669572880922054, 'y': -118.98955452806044}},
 {'data': {'id': '5', 'layer': 1, 'dim': 2},
  'position': {'x': 49.03239353727031, 'y': -99.92655038354535}},
 {'data': {'id': '6', 'layer': 1, 'dim': 2},
  'position': {'x': -24.079728052410907, 'y': 26.75216510196429}},
 {'data': {'id': '7', 'layer': 1, 'dim': 4},
  'position': {'x': -19.06590492451627, 'y': -114.8670083006786}},
 {'data': {'id': '8', 'layer': 1, 'dim': 2},
  'position': {'x': 3.9436616616810296, 'y': 75.1846901103

In [73]:
basegraph = layers[-2]["graph"]
for it in range(100):
    basegraph.synchronous_iteration()
    energy = basegraph.energy_map(include_priors=True, include_factors=True)
    print(f"Iter {it+1:03d} | Energy = {energy:.6f}")


Iter 001 | Energy = 40.090725
Iter 002 | Energy = 19.865964
Iter 003 | Energy = 21.135094
Iter 004 | Energy = 20.097458
Iter 005 | Energy = 19.868294
Iter 006 | Energy = 19.827337
Iter 007 | Energy = 19.867221
Iter 008 | Energy = 19.892125
Iter 009 | Energy = 19.905667
Iter 010 | Energy = 19.910118
Iter 011 | Energy = 19.912568
Iter 012 | Energy = 19.913763
Iter 013 | Energy = 19.914407
Iter 014 | Energy = 19.914782
Iter 015 | Energy = 19.914986
Iter 016 | Energy = 19.915126
Iter 017 | Energy = 19.915205
Iter 018 | Energy = 19.915239
Iter 019 | Energy = 19.915259
Iter 020 | Energy = 19.915270
Iter 021 | Energy = 19.915276
Iter 022 | Energy = 19.915279
Iter 023 | Energy = 19.915281
Iter 024 | Energy = 19.915282
Iter 025 | Energy = 19.915282
Iter 026 | Energy = 19.915282
Iter 027 | Energy = 19.915282
Iter 028 | Energy = 19.915283
Iter 029 | Energy = 19.915283
Iter 030 | Energy = 19.915283
Iter 031 | Energy = 19.915283
Iter 032 | Energy = 19.915283
Iter 033 | Energy = 19.915283
Iter 034 |

In [74]:
supergraph = layers[-1]["graph"]

# 找到所有 super_prior factors
prior_factors = [f for f in supergraph.factors if getattr(f, "type", "") == "super_prior"]

# 对 super_prior factors 的邻居变量
prior_vars = []
for f in prior_factors:
    for v in f.adj_var_nodes:
        if v not in prior_vars:
            prior_vars.append(v)

# 先对 super_prior factors 做一次消息传递 & belief 更新
supergraph.compute_all_messages(prior_factors, local_relin=True)
supergraph.update_all_beliefs(prior_vars)
supergraph.var_nodes[0].belief.eta
supergraph.var_nodes[0].belief.lam

array([[0., 0.],
       [0., 0.]])

In [75]:
supergraph.var_nodes[0].belief.lam.shape

(2, 2)

In [76]:
print(np.linalg.solve(basegraph.var_nodes[0].belief.lam, basegraph.var_nodes[0].belief.eta))
print(np.linalg.solve(basegraph.var_nodes[1].belief.lam, basegraph.var_nodes[1].belief.eta))

[1.00071248 0.38491876]
[-24.00720078   5.06173455]


In [78]:
supergraph.var_nodes[0].belief.lam

array([[0., 0.],
       [0., 0.]])

In [77]:
np.linalg.solve(supergraph.var_nodes[0].belief.lam, supergraph.var_nodes[0].belief.eta)


LinAlgError: Singular matrix

In [ ]:
supergraph = layers[-1]["graph"]

# 找到所有 super_prior factors
prior_factors = [f for f in supergraph.factors if getattr(f, "type", "") == "super_prior"]

# 对 super_prior factors 的邻居变量
prior_vars = []
for f in prior_factors:
    for v in f.adj_var_nodes:
        if v not in prior_vars:
            prior_vars.append(v)

# 先对 super_prior factors 做一次消息传递 & belief 更新
supergraph.compute_all_messages(prior_factors, local_relin=True)
supergraph.update_all_beliefs(prior_vars)

for it in range(100):
    supergraph.synchronous_iteration()
    energy = supergraph.energy_map(include_priors=True, include_factors=True)
    print(f"Iter {it+1:03d} | Energy = {energy:.6f}")

Iter 001 | Energy = 24.363011
Iter 002 | Energy = 24.363011
Iter 003 | Energy = 24.363011
Iter 004 | Energy = 24.363011
Iter 005 | Energy = 24.363011
Iter 006 | Energy = 24.363011
Iter 007 | Energy = 24.363011
Iter 008 | Energy = 24.363011
Iter 009 | Energy = 24.363011
Iter 010 | Energy = 24.363011
Iter 011 | Energy = 24.363011
Iter 012 | Energy = 24.363011
Iter 013 | Energy = 24.363011
Iter 014 | Energy = 24.363011
Iter 015 | Energy = 24.363011
Iter 016 | Energy = 24.363011
Iter 017 | Energy = 24.363011
Iter 018 | Energy = 24.363011
Iter 019 | Energy = 24.363011
Iter 020 | Energy = 24.363011
Iter 021 | Energy = 24.363011
Iter 022 | Energy = 24.363011
Iter 023 | Energy = 24.363011
Iter 024 | Energy = 24.363011
Iter 025 | Energy = 24.363011
Iter 026 | Energy = 24.363011
Iter 027 | Energy = 24.363011
Iter 028 | Energy = 24.363011
Iter 029 | Energy = 24.363011
Iter 030 | Energy = 24.363011
Iter 031 | Energy = 24.363011
Iter 032 | Energy = 24.363011
Iter 033 | Energy = 24.363011
Iter 034 |